# `ChatPromptBuilder`
- Enable GPU in Docker [https://github.com/ollama/ollama/blob/main/docs/docker.md](https://github.com/ollama/ollama/blob/main/docs/docker.md):
  ```bash
  curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey \
      | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
  curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list \
      | sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' \
      | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list
  sudo apt-get update

  sudo apt-get install -y nvidia-container-toolkit

  sudo nvidia-ctk runtime configure --runtime=docker

  sudo systemctl restart docker
  ```

- Run Ollama in Docker integrated with GPU:
  ```bash
  docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
  docker exec ollama ollama pull hf.co/janhq/Vistral-7b-Chat-GGUF:Q5_K_M
  docker exec ollama ollama pull hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0
  ```

In [1]:
%load_ext autoreload
%autoreload 2

## On its own

In [2]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage

In [3]:
template = [ChatMessage.from_user("Translate to {{ target_language }}. Context: {{ snippet }}; Translation:")]
builder = ChatPromptBuilder(template=template)
res = builder.run(target_language="spanish", snippet="I can't speak spanish.")

In [4]:
res

{'prompt': [ChatMessage(content="Translate to spanish. Context: I can't speak spanish.; Translation:", role=<ChatRole.USER: 'user'>, name=None, meta={})]}

### Overriding static template at runtime

In [5]:
summary_template = [ChatMessage.from_user("Translate to {{ target_language }} and summarize. Context: {{ snippet }}; Summary:")]
res = builder.run(target_language="spanish", snippet="I can't speak spanish.", template=summary_template)

In [6]:
res

{'prompt': [ChatMessage(content="Translate to spanish and summarize. Context: I can't speak spanish.; Summary:", role=<ChatRole.USER: 'user'>, name=None, meta={})]}

## In pipeline

In [7]:
from haystack_integrations.components.generators.ollama import OllamaChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack import Pipeline

In [8]:
# no parameter init, we don't use any runtime template variables
prompt_builder = ChatPromptBuilder()
llm = OllamaChatGenerator(
    model="hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://localhost:11434",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9})

In [9]:
pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [10]:
location = "Thành phố Hồ Chi Minh"
language = "Việt Nam"
system_message = ChatMessage.from_system("Bạn là trợ lý cung cấp thông tin cho khách du lịch tại {{language}}")
messages = [system_message, ChatMessage.from_user("Tell me about {{location}}")]

res = pipe.run(data={"prompt_builder": {"template_variables": {"location": location, "language": language},
                                    "template": messages}})

print(res)

Thành phố Hồ Chí Minh, thường được gọi là Sài Gòn, là một trong những thành phố lớn nhất Việt Nam. Nó nằm ở phía nam của đất nước và đóng vai trò quan trọng như trung tâm thương mại, kinh tế và văn hoá chính trị. Thành phố nổi tiếng với những tòa nhà chọc trời hiện đại cũng như sự pha trộn giữa các khu vực lịch sử cổ kính và phát triển mới mẻ
Thành phố Hồ Chí Minh có dân số hơn 8 triệu người và được biết đến{'llm': {'replies': [ChatMessage(content='Thành phố Hồ Chí Minh, thường được gọi là Sài Gòn, là một trong những thành phố lớn nhất Việt Nam. Nó nằm ở phía nam của đất nước và đóng vai trò quan trọng như trung tâm thương mại, kinh tế và văn hoá chính trị. Thành phố nổi tiếng với những tòa nhà chọc trời hiện đại cũng như sự pha trộn giữa các khu vực lịch sử cổ kính và phát triển mới mẻ\nThành phố Hồ Chí Minh có dân số hơn 8 triệu người và được biết đến', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={})], 'meta': [{'model': 'hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0', 'created_a

Now I can ask about the weather in HoChiMinh City.

In [11]:
location = "Thành phố Hồ Chi Minh"

user_message1 = ChatMessage.from_user("Thời tiết cùa {{location}} như thế nào trong {{day_count}} ngày tới?")
messages = [system_message, user_message1]
res = pipe.run(data={"prompt_builder": {"template_variables": {"location": location, "day_count": "5"},
                                    "template": messages}})

print(res)

Để tôi kiểm tra dự báo thời tiết cho bạn. Theo dự đoán, nhiệt độ sẽ ở mức từ 26 đến 31 C với phần lớn các ngày có mưa nhỏ hoặc không mưa vào những ngày sắp tới. Thời tiết được mong đợi là nắng đẹp và hơi ấm áp. Hãy nhớ mang theo một chiếc ô nếu cần thiết. Tôi hy vọng điều này hữu ích!
{'llm': {'replies': [ChatMessage(content='Để tôi kiểm tra dự báo thời tiết cho bạn. Theo dự đoán, nhiệt độ sẽ ở mức từ 26 đến 31 C với phần lớn các ngày có mưa nhỏ hoặc không mưa vào những ngày sắp tới. Thời tiết được mong đợi là nắng đẹp và hơi ấm áp. Hãy nhớ mang theo một chiếc ô nếu cần thiết. Tôi hy vọng điều này hữu ích!\n', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={})], 'meta': [{'model': 'hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0', 'created_at': '2024-12-29T08:21:55.576232262Z', 'done': False, 'done_reason': None, 'total_duration': None, 'load_duration': None, 'prompt_eval_count': None, 'prompt_eval_duration': None, 'eval_count': None, 'eval_duration': None, 'role': 'assistant'}]}}


In [12]:
location = "Thành phố Hồ Chi Minh"

user_message2 = ChatMessage.from_user("Có món ăn nào là đặc sản ở {{location}} không?")
messages = [system_message, user_message2]
res = pipe.run(data={"prompt_builder": {"template_variables": {"location": location, "day_count": "5"},
                                    "template": messages}})

print(res)

Đúng, có nhiều món ăn ngon và đa dạng ở thành phố HCM. Một số trong những cái phổ biến nhất bao gồm:
1. Phở - một bát phở bò hoặc gà ấm áp được phục vụ với mì gạo mỏng, nước dùng thơm ngon và các loại thảo mộc tươi.
2. Bún Bò Huế - súp đậm đà làm từ bún gạo, thịt bò, chả giò và rau củ ăn kèm mắm tôm cay.
3. Cơm tấm Sài Gòn - cơm thơm nấu{'llm': {'replies': [ChatMessage(content='Đúng, có nhiều món ăn ngon và đa dạng ở thành phố HCM. Một số trong những cái phổ biến nhất bao gồm:\n1. Phở - một bát phở bò hoặc gà ấm áp được phục vụ với mì gạo mỏng, nước dùng thơm ngon và các loại thảo mộc tươi.\n2. Bún Bò Huế - súp đậm đà làm từ bún gạo, thịt bò, chả giò và rau củ ăn kèm mắm tôm cay.\n3. Cơm tấm Sài Gòn - cơm thơm nấu', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={})], 'meta': [{'model': 'hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0', 'created_at': '2024-12-29T08:22:01.055821272Z', 'done': False, 'done_reason': None, 'total_duration': None, 'load_duration': None, 'prompt_eval_count': 